<a href="https://colab.research.google.com/github/MMoronto/pytorch_exercises/blob/master/Peutingar_datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Data Dictionary

In [2]:
pip install pytorch-forecasting

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.4/141.4 KB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.8/30.8 MB 46.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 827.8/827.8 KB 50.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.2/308.2 KB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.0/81.0 KB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 KB 37.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 57.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 KB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.2/147.2 KB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 KB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━

##Load data

In [3]:
from google.colab import drive
drive.mount('/content/drive')

path1='/content/drive/MyDrive/Colab Notebooks/infra_data/Salient_cement_stats.xls'
path2='/content/drive/MyDrive/Colab Notebooks/infra_data/Salient_cement_stats.csv'
fob=open(path2,)
headings=next(fob)
for rec in fob:
  Cement3=rec.split(',')
  print(Cement3)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
['"SALIENT CEMENT STATISTICS1', ' 2 "', '', '', '', '', '', '', '\n']
['', '', '', '', '', '', '', '\n']
['(Thousand metric tons unless otherwise specified)', '', '', '', '', '', '', '\n']
['', '', '', '', '', '', '', '\n']
['', '', '', '2014', '2015', '2016', '2017', '2018\n']
['United States:', '', '', '', '', '', '', '\n']
['Production:', '', '', '', '', '', '', '\n']
['Cement3', '', '', '"82', '535"', '"84', '405"', '"84', '695"', '"86', '356"', '"86', '400"\n']
['Clinker', '', '', '"74', '372"', '"76', '043"', '"75', '633"', '"76', '678"', '"77', '112"\n']
['"Shipments from mills and terminals:3', ' 4"', '', '', '', '', '', '', '\n']
['Quantity', '', '', '"88', '900"', '"92', '000"', '"94', '300"', '"96', '900"', '"98', '500"\n']
['Value5', 'thousand dollars', '', '"8', '940', '000"', '"9', '800', '000"', '"10', '500', '000"', '"11', '300', '000"', '"11'

In [4]:
import os
import warnings

warnings.filterwarnings("ignore")  # avoid printing out absolute paths

os.chdir("../../..")

import copy
from pathlib import Path
import warnings

import numpy as np
import pandas as pd
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger
import torch

from pytorch_forecasting import Baseline, TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import SMAPE, PoissonLoss, QuantileLoss
from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters


In [5]:
# Create a series and let pandas create a default integer index:
s = pd.Series([1, 3, 5, np.nan, 6, 8])
s

0    1.0
1    3.0
2    5.0
3    NaN
4    6.0
5    8.0
dtype: float64

In [6]:
from pandas.io.stata import DatetimeIndex
# Create a DataFrame by passing a NumPy array with a dateline index using `date_range()` & labeled columns:
dates = pd.date_range("20230101", periods=8)

dates

# DatetimeIndex(['2013-01-01', '2013-01-02', '2013-01-03', '2013-01-04', '2013-01-05', '2013-01-06'], dtype='datatime64[ns]', freq='D')

df = pd.DataFrame(np.random.randn(8, 6), index=dates, columns=list("ABCDEF"))

df 

,A,B,C,D,E,F
2023-01-01,-0.917697,-1.248071,-0.178938,1.024680,1.161252,0.508894
2023-01-02,-0.842315,-0.018534,-0.420676,0.522747,0.631991,1.133723
2023-01-03,-0.878407,-0.538345,-0.121613,-1.265034,-1.701057,-0.763982
2023-01-04,-0.805528,0.066429,-0.905822,-1.180490,1.801049,0.906112
2023-01-05,-2.084088,0.061432,0.352216,0.592021,1.573709,-0.360456
2023-01-06,-0.179819,0.048646,-2.185305,-1.001107,-0.313357,1.589739
2023-01-07,1.010149,1.469951,-2.305192,0.337235,0.523993,1.563506
2023-01-08,-0.612347,-0.459967,0.518692,-0.956754,-0.532372,2.566974


In [7]:
# Create a `DataFrame` by passing a dictionary of objects that can be converted into a series-like structure:

df2 = pd.DataFrame(
    {
        "A": 1.0,
        "B": pd.Timestamp("20130102"),
        "C": pd.Series(1, index=list(range(4)), dtype="float32"),
        "D": np.array([3] * 4, dtype="int32"),
        "E": pd.Categorical(["test", "train", "test", "train"]),
        "F": "foo",
    }
)

df2

,A,B,C,D,E,F
0,1.0,2013-01-02,1.0,3,test,foo
1,1.0,2013-01-02,1.0,3,train,foo
2,1.0,2013-01-02,1.0,3,test,foo
3,1.0,2013-01-02,1.0,3,train,foo


In [8]:
df2.dtypes

A           float64
B    datetime64[ns]
C           float32
D             int32
E          category
F            object
dtype: object

In [9]:
df2.D

0    3
1    3
2    3
3    3
Name: D, dtype: int32

##Viewing data

In [10]:
df.head()

,A,B,C,D,E,F
2023-01-01,-0.917697,-1.248071,-0.178938,1.024680,1.161252,0.508894
2023-01-02,-0.842315,-0.018534,-0.420676,0.522747,0.631991,1.133723
2023-01-03,-0.878407,-0.538345,-0.121613,-1.265034,-1.701057,-0.763982
2023-01-04,-0.805528,0.066429,-0.905822,-1.180490,1.801049,0.906112
2023-01-05,-2.084088,0.061432,0.352216,0.592021,1.573709,-0.360456


In [11]:
df.tail()

,A,B,C,D,E,F
2023-01-04,-0.805528,0.066429,-0.905822,-1.180490,1.801049,0.906112
2023-01-05,-2.084088,0.061432,0.352216,0.592021,1.573709,-0.360456
2023-01-06,-0.179819,0.048646,-2.185305,-1.001107,-0.313357,1.589739
2023-01-07,1.010149,1.469951,-2.305192,0.337235,0.523993,1.563506
2023-01-08,-0.612347,-0.459967,0.518692,-0.956754,-0.532372,2.566974


In [12]:
df.columns

Index(['A', 'B', 'C', 'D', 'E', 'F'], dtype='object')

In [13]:
df.index

DatetimeIndex(['2023-01-01', '2023-01-02', '2023-01-03', '2023-01-04',
               '2023-01-05', '2023-01-06', '2023-01-07', '2023-01-08'],
              dtype='datetime64[ns]', freq='D')

`DataFrame.to_numpy()` gives a NumPy representation of the underlying data. 

In [14]:
df.to_numpy()

array([[-0.91769675, -1.24807095, -0.17893776,  1.02468009,  1.16125227,
         0.50889371],
       [-0.84231515, -0.01853438, -0.42067579,  0.52274742,  0.63199148,
         1.13372303],
       [-0.87840679, -0.53834478, -0.12161316, -1.26503428, -1.70105743,
        -0.76398175],
       [-0.80552773,  0.06642917, -0.90582172, -1.18049033,  1.80104935,
         0.9061115 ],
       [-2.08408778,  0.06143199,  0.35221606,  0.59202117,  1.57370859,
        -0.36045588],
       [-0.17981876,  0.04864634, -2.18530482, -1.00110737, -0.31335732,
         1.5897391 ],
       [ 1.01014865,  1.46995134, -2.30519237,  0.33723541,  0.5239928 ,
         1.56350571],
       [-0.61234677, -0.45996731,  0.51869173, -0.9567545 , -0.53237156,
         2.56697402]])

In [15]:
df2.to_numpy()

array([[1.0, Timestamp('2013-01-02 00:00:00'), 1.0, 3, 'test', 'foo'],
       [1.0, Timestamp('2013-01-02 00:00:00'), 1.0, 3, 'train', 'foo'],
       [1.0, Timestamp('2013-01-02 00:00:00'), 1.0, 3, 'test', 'foo'],
       [1.0, Timestamp('2013-01-02 00:00:00'), 1.0, 3, 'train', 'foo']],
      dtype=object)

In [16]:
df.describe()

,A,B,C,D,E,F
count,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000
mean,-0.663756,-0.077307,-0.655830,-0.240838,0.393151,0.893064
std,0.863438,0.773019,1.074773,0.943768,1.182098,1.086252
min,-2.084088,-1.248071,-2.305192,-1.265034,-1.701057,-0.763982
25%,-0.888229,-0.479562,-1.225692,-1.045953,-0.368111,0.291556
50%,-0.823921,0.015056,-0.299807,-0.309760,0.577992,1.019917
75%,-0.504215,0.062681,-0.003156,0.540066,1.264366,1.570064
max,1.010149,1.469951,0.518692,1.024680,1.801049,2.566974


In [17]:
df

,A,B,C,D,E,F
2023-01-01,-0.917697,-1.248071,-0.178938,1.024680,1.161252,0.508894
2023-01-02,-0.842315,-0.018534,-0.420676,0.522747,0.631991,1.133723
2023-01-03,-0.878407,-0.538345,-0.121613,-1.265034,-1.701057,-0.763982
2023-01-04,-0.805528,0.066429,-0.905822,-1.180490,1.801049,0.906112
2023-01-05,-2.084088,0.061432,0.352216,0.592021,1.573709,-0.360456
2023-01-06,-0.179819,0.048646,-2.185305,-1.001107,-0.313357,1.589739
2023-01-07,1.010149,1.469951,-2.305192,0.337235,0.523993,1.563506
2023-01-08,-0.612347,-0.459967,0.518692,-0.956754,-0.532372,2.566974


In [18]:
df.T

,2023-01-01,2023-01-02,2023-01-03,2023-01-04,2023-01-05,2023-01-06,2023-01-07,2023-01-08
A,-0.917697,-0.842315,-0.878407,-0.805528,-2.084088,-0.179819,1.010149,-0.612347
B,-1.248071,-0.018534,-0.538345,0.066429,0.061432,0.048646,1.469951,-0.459967
C,-0.178938,-0.420676,-0.121613,-0.905822,0.352216,-2.185305,-2.305192,0.518692
D,1.024680,0.522747,-1.265034,-1.180490,0.592021,-1.001107,0.337235,-0.956754
E,1.161252,0.631991,-1.701057,1.801049,1.573709,-0.313357,0.523993,-0.532372
F,0.508894,1.133723,-0.763982,0.906112,-0.360456,1.589739,1.563506,2.566974


In [19]:
df.sort_index(axis=1, ascending=False)

,F,E,D,C,B,A
2023-01-01,0.508894,1.161252,1.024680,-0.178938,-1.248071,-0.917697
2023-01-02,1.133723,0.631991,0.522747,-0.420676,-0.018534,-0.842315
2023-01-03,-0.763982,-1.701057,-1.265034,-0.121613,-0.538345,-0.878407
2023-01-04,0.906112,1.801049,-1.180490,-0.905822,0.066429,-0.805528
2023-01-05,-0.360456,1.573709,0.592021,0.352216,0.061432,-2.084088
2023-01-06,1.589739,-0.313357,-1.001107,-2.185305,0.048646,-0.179819
2023-01-07,1.563506,0.523993,0.337235,-2.305192,1.469951,1.010149
2023-01-08,2.566974,-0.532372,-0.956754,0.518692,-0.459967,-0.612347


In [20]:
df.sort_values(by="B")

,A,B,C,D,E,F
2023-01-01,-0.917697,-1.248071,-0.178938,1.024680,1.161252,0.508894
2023-01-03,-0.878407,-0.538345,-0.121613,-1.265034,-1.701057,-0.763982
2023-01-08,-0.612347,-0.459967,0.518692,-0.956754,-0.532372,2.566974
2023-01-02,-0.842315,-0.018534,-0.420676,0.522747,0.631991,1.133723
2023-01-06,-0.179819,0.048646,-2.185305,-1.001107,-0.313357,1.589739
2023-01-05,-2.084088,0.061432,0.352216,0.592021,1.573709,-0.360456
2023-01-04,-0.805528,0.066429,-0.905822,-1.180490,1.801049,0.906112
2023-01-07,1.010149,1.469951,-2.305192,0.337235,0.523993,1.563506


##Getting

In [21]:
# Select a single column, which yields a Series, equivalent to 'df.A'
df["A"]

2023-01-01   -0.917697
2023-01-02   -0.842315
2023-01-03   -0.878407
2023-01-04   -0.805528
2023-01-05   -2.084088
2023-01-06   -0.179819
2023-01-07    1.010149
2023-01-08   -0.612347
Freq: D, Name: A, dtype: float64

In [22]:
df[0:3]

,A,B,C,D,E,F
2023-01-01,-0.917697,-1.248071,-0.178938,1.024680,1.161252,0.508894
2023-01-02,-0.842315,-0.018534,-0.420676,0.522747,0.631991,1.133723
2023-01-03,-0.878407,-0.538345,-0.121613,-1.265034,-1.701057,-0.763982


##Selection by label

In [23]:
df.loc[dates[1]]

A   -0.842315
B   -0.018534
C   -0.420676
D    0.522747
E    0.631991
F    1.133723
Name: 2023-01-02 00:00:00, dtype: float64

In [24]:
#Selecting on a multi-axis by label:
df.loc[:, ["A", "D"]]

,A,D
2023-01-01,-0.917697,1.024680
2023-01-02,-0.842315,0.522747
2023-01-03,-0.878407,-1.265034
2023-01-04,-0.805528,-1.180490
2023-01-05,-2.084088,0.592021
2023-01-06,-0.179819,-1.001107
2023-01-07,1.010149,0.337235
2023-01-08,-0.612347,-0.956754


In [25]:
# Showing label slicing, both endpoints are included:
df.loc["20230104":"20230106", ["A", "D"]]

,A,D
2023-01-04,-0.805528,-1.180490
2023-01-05,-2.084088,0.592021
2023-01-06,-0.179819,-1.001107


In [26]:
# Reduction in the dimensions of the returned object:
df.loc["20230104", ["A", "D"]]

A   -0.805528
D   -1.180490
Name: 2023-01-04 00:00:00, dtype: float64

In [27]:
# For getting a scalar value
df.loc[dates[1], "D"]

0.5227474177055662

In [28]:
# For getting fast access to a scalar value
df.at[dates[1], "D"]

0.5227474177055662

##Selection by position

In [29]:
df.iloc[3]

A   -0.805528
B    0.066429
C   -0.905822
D   -1.180490
E    1.801049
F    0.906112
Name: 2023-01-04 00:00:00, dtype: float64

In [30]:
df.iloc[3:5, 0:2]

,A,B
2023-01-04,-0.805528,0.066429
2023-01-05,-2.084088,0.061432


In [31]:
df.iloc[[1, 2, 4], [0, 2]]

,A,C
2023-01-02,-0.842315,-0.420676
2023-01-03,-0.878407,-0.121613
2023-01-05,-2.084088,0.352216


In [32]:
# For slicing rows explicitly
df.iloc[1:3, :]

,A,B,C,D,E,F
2023-01-02,-0.842315,-0.018534,-0.420676,0.522747,0.631991,1.133723
2023-01-03,-0.878407,-0.538345,-0.121613,-1.265034,-1.701057,-0.763982


In [33]:
# For slicing columns explicitly
df.iloc[:, 1:3]

,B,C
2023-01-01,-1.248071,-0.178938
2023-01-02,-0.018534,-0.420676
2023-01-03,-0.538345,-0.121613
2023-01-04,0.066429,-0.905822
2023-01-05,0.061432,0.352216
2023-01-06,0.048646,-2.185305
2023-01-07,1.469951,-2.305192
2023-01-08,-0.459967,0.518692


In [34]:
# For getting values explicitly
df.iloc[1, 1]

-0.018534380230896222

In [35]:
# For getting fast access to scalar (equivalent to the prior method)
df.iat[1, 1]

-0.018534380230896222

## Boolean indexing
Using a single column's values to select data:

In [36]:
df[df["A"] > 0]

,A,B,C,D,E,F
2023-01-07,1.010149,1.469951,-2.305192,0.337235,0.523993,1.563506


In [37]:
# Selecting values from a DataFrame where a boolean condition is met:
df[df > 0]

,A,B,C,D,E,F
2023-01-01,NaN,NaN,NaN,1.024680,1.161252,0.508894
2023-01-02,NaN,NaN,NaN,0.522747,0.631991,1.133723
2023-01-03,NaN,NaN,NaN,NaN,NaN,NaN
2023-01-04,NaN,0.066429,NaN,NaN,1.801049,0.906112
2023-01-05,NaN,0.061432,0.352216,0.592021,1.573709,NaN
2023-01-06,NaN,0.048646,NaN,NaN,NaN,1.589739
2023-01-07,1.010149,1.469951,NaN,0.337235,0.523993,1.563506
2023-01-08,NaN,NaN,0.518692,NaN,NaN,2.566974


In [50]:
# Using the isin() method for filtering:
df2 = df.copy()

df2["E"] = ["one", "one", "two", "three", "four", "three", "five", "four"]

df2

,A,B,C,D,E,F
2023-01-01,0.000000,0.000000,-0.178938,5,one,NaN
2023-01-02,-0.842315,-0.018534,-0.420676,5,one,NaN
2023-01-03,-0.878407,-0.538345,-0.121613,5,two,NaN
2023-01-04,-0.805528,0.066429,-0.905822,5,three,NaN
2023-01-05,-2.084088,0.061432,0.352216,5,four,NaN
2023-01-06,-0.179819,0.048646,-2.185305,5,three,NaN
2023-01-07,1.010149,1.469951,-2.305192,5,five,NaN
2023-01-08,-0.612347,-0.459967,0.518692,5,four,NaN


In [51]:
df2["E"] = ["one", "one", "two", "three", "four", "three", "five", "four"]

df2[df2["E"].isin(["two", "four"])]

,A,B,C,D,E,F
2023-01-03,-0.878407,-0.538345,-0.121613,5,two,NaN
2023-01-05,-2.084088,0.061432,0.352216,5,four,NaN
2023-01-08,-0.612347,-0.459967,0.518692,5,four,NaN


## Setting

Setting a new column automatically aligns the data by the indexes:

In [41]:
s1 = pd.Series([1, 2, 3, 4, 5, 6], index=pd.date_range("20130102", periods=6))
s1

2013-01-02    1
2013-01-03    2
2013-01-04    3
2013-01-05    4
2013-01-06    5
2013-01-07    6
Freq: D, dtype: int64

In [42]:
df["F"] = s1

In [43]:
# Setting values by label:
df.at[dates[0], "A"] = 0

In [44]:
# Setting values by position:
df.iat[0, 1] = 0

In [45]:
# Setting by assigning with a NumPy array:
df.loc[:, "D"] = np.array([5] * len(df))

In [46]:
# The result of the prior setting operations:
df

,A,B,C,D,E,F
2023-01-01,0.000000,0.000000,-0.178938,5,1.161252,NaN
2023-01-02,-0.842315,-0.018534,-0.420676,5,0.631991,NaN
2023-01-03,-0.878407,-0.538345,-0.121613,5,-1.701057,NaN
2023-01-04,-0.805528,0.066429,-0.905822,5,1.801049,NaN
2023-01-05,-2.084088,0.061432,0.352216,5,1.573709,NaN
2023-01-06,-0.179819,0.048646,-2.185305,5,-0.313357,NaN
2023-01-07,1.010149,1.469951,-2.305192,5,0.523993,NaN
2023-01-08,-0.612347,-0.459967,0.518692,5,-0.532372,NaN


In [47]:
# A where operation with setting:

df2 = df.copy()

df2[df2 > 0] = -df2

df2

,A,B,C,D,E,F
2023-01-01,0.000000,0.000000,-0.178938,-5,-1.161252,NaN
2023-01-02,-0.842315,-0.018534,-0.420676,-5,-0.631991,NaN
2023-01-03,-0.878407,-0.538345,-0.121613,-5,-1.701057,NaN
2023-01-04,-0.805528,-0.066429,-0.905822,-5,-1.801049,NaN
2023-01-05,-2.084088,-0.061432,-0.352216,-5,-1.573709,NaN
2023-01-06,-0.179819,-0.048646,-2.185305,-5,-0.313357,NaN
2023-01-07,-1.010149,-1.469951,-2.305192,-5,-0.523993,NaN
2023-01-08,-0.612347,-0.459967,-0.518692,-5,-0.532372,NaN


##Missing data
pandas primarily uses the value `np.nan` to represent missing data. It is by default not included in computations.

Reindexing allows you to change/add/delete the index on a specified axis. This returns a copy of the data:

In [52]:
df1 = df.reindex(index=dates[0:4], columns=list(df.columns) + ["E"])

df1.loc[dates[0] : dates[1], "E"] = 1

df1

,A,B,C,D,E,F,E
2023-01-01,0.000000,0.000000,-0.178938,5,1.000000,NaN,1.000000
2023-01-02,-0.842315,-0.018534,-0.420676,5,1.000000,NaN,1.000000
2023-01-03,-0.878407,-0.538345,-0.121613,5,-1.701057,NaN,-1.701057
2023-01-04,-0.805528,0.066429,-0.905822,5,1.801049,NaN,1.801049


`DataFrame.dropna()` drops any rows that have missing data:

In [53]:
df1.dropna(how="any")

,A,B,C,D,E,F,E


`DataFrame.fillna()` fills missing data:

In [55]:
df1.fillna(value=0)

,A,B,C,D,E,F,E
2023-01-01,0.000000,0.000000,-0.178938,5,1.000000,0.0,1.000000
2023-01-02,-0.842315,-0.018534,-0.420676,5,1.000000,0.0,1.000000
2023-01-03,-0.878407,-0.538345,-0.121613,5,-1.701057,0.0,-1.701057
2023-01-04,-0.805528,0.066429,-0.905822,5,1.801049,0.0,1.801049


`isna()` gets the boolean mask where values are `nan`:

In [56]:
pd.isna(df1)

,A,B,C,D,E,F,E
2023-01-01,False,False,False,False,False,True,False
2023-01-02,False,False,False,False,False,True,False
2023-01-03,False,False,False,False,False,True,False
2023-01-04,False,False,False,False,False,True,False
